In this notebook we will learn how we could use a neural network to predict cancer based on medical images. 

### Start by importing packages


In [ ]:
from sklearn.neural_network import MLPClassifier
from sklearn.datasets import make_moons, load_breast_cancer
from sklearn.model_selection import train_test_split
from matplotlib import pyplot as plt

### Load the data

Again we will use a dataset built-in to Sklearn that includes data related to diagnosing breast cancer.

In [ ]:
cancer = load_breast_cancer()

# print('Dataset raw object', cancer)
print('Dataset description', cancer['DESCR'])

## Split into our training and testing XY sets

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(cancer.data, cancer.target, random_state=0)

## Exercise 1: Scale the data

The Multilayer Perceptron (MLP) approach is one of the few that doesn't automatically scale the data, so let's do that. Here we will use Numpy to do it manually, though there are alternative built-in methods within scikit-learn.

In the code block below, use `X_train.mean(axis=0)` and similar functions to scale ALL of the X variables so that they have mean 0 and standard deviation 1. HINT: X_train and others are numpy arrays and so you can use fast raster math, e.g., `X_train - mean_on_train`.

In [ ]:
# Exercies 1 Code

# Using numpy functions, compute the mean value per feature on the training set and the STD.

## Create the MLP model object and fit it

Using this new scaled training data, we are ready to define a Neural Net, Known here as a Multi-Layer-Perceptron (MLP) classifier. Because this next line hides away millions of other lines of code, you may want to explore it. In VS Code, you can navigate to a function's definition by placing your cursor in the function and press f-12. Try it in the cell below on the `MLPClassifier` code! The best documentation is often the code itself.

In [ ]:
mlp = MLPClassifier(random_state=0)

# Now fit it with the scaled X and y TRAINING data.
mlp.fit(X_train_scaled, y_train)

print(mlp)

## Assess the fit

Now we assess MLP's accuracy on the TRAINING and the TESTING data.

Notice here also I'm introducing another convenient way of combining strings and numbers. The {:.2f} specifies a placeholder for a 2-digit representation of a floating point number. The Format method then places that floating point value into that placeholder.

In [ ]:
score_train = mlp.score(X_train_scaled, y_train)
score_test = mlp.score(X_test_scaled, y_test)

print("Accuracy on training set: {:.3f}".format(score_train))
print("Accuracy on test set: {:.3f}".format(mlp.score(X_test_scaled, y_test)))

## Plot the inputs and hidden layers of the neural net

It can be hard perhaps to visualize what exaclty the neural net looks like (there is no coefficients table to simply look at). But here, it is small enough to actually visualize the coefficients within the network.

Below, we plot the coeffs_ array to see it.

In [ ]:
plt.figure(figsize=(20, 5))
plt.imshow(mlp.coefs_[0], interpolation='none', cmap='viridis')
plt.yticks(range(30), cancer.feature_names)
plt.xlabel("Columns in weight matrix")
plt.ylabel("Input feature")
plt.colorbar()
plt.show()

## Exercies 5.1.2: Understanding which features matter most

One of the massive challenges in Neural Nets is understanding why exactly it makes the predictions it does. Can you identify which input feature shows the largest positive effect on on cancer diagnosis?

You probably can't make heads or tails of it. Let's create a greatly simplified version of our neural network to try to see if we can understand it.

Specifically create a new `MLPClassifier` but this time make it have only a single hidden layer. Hint: use f-12 on the MLPClassifier code to see it's documentation and figure out what new input variable you sohuld specify when calling `mlp = MLPClassifier( .......  )`. Plot the output coefficients just like above. With only a single layer, the variables become somewhat more interpretable. 

Which variable now seems to have the largest positive impact?

In [ ]:
# Excercise 5.1.2 workspace

### Other approaches to understanding neural nets

- Let's apply our new MLPClassifier model type to our hand-written digits dataset.
  - Here, though, I want to highight a few other tools used in the ML workflow.
    - make_pipeline
    - StandardScaler

In [ ]:
import warnings

import matplotlib.pyplot as plt

from sklearn.datasets import fetch_openml
from sklearn.exceptions import ConvergenceWarning
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier

### Use a fetch_openml helper function to get the MNIST dataset

In [ ]:
# Load data from https://www.openml.org/d/554
X, y = fetch_openml(
    "mnist_784", version=1, return_X_y=True, as_frame=False, parser="pandas"
)

### Scale the X variables

In [ ]:
X = X / 255.0

### Split data into train partition and test partition

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0, test_size=0.7)

In [ ]:
print('Look at the data')
print(X_train.shape, y_train.shape)
print(X_train, y_train)

In [ ]:
mlp = MLPClassifier(
    hidden_layer_sizes=(40,),
    max_iter=8,
    alpha=1e-4,
    solver="sgd",
    verbose=10,
    random_state=1,
    learning_rate_init=0.2,
)

# this example won't converge because of resource usage constraints on
# our Continuous Integration infrastructure, so we catch the warning and
# ignore it here
with warnings.catch_warnings():
    warnings.filterwarnings("ignore", category=ConvergenceWarning, module="sklearn")
    mlp.fit(X_train, y_train)

print("Training set score: %f" % mlp.score(X_train, y_train))
print("Test set score: %f" % mlp.score(X_test, y_test))

In [ ]:
fig, axes = plt.subplots(4, 4)
# use global min / max to ensure all weights are shown on the same scale
vmin, vmax = mlp.coefs_[0].min(), mlp.coefs_[0].max()
for coef, ax in zip(mlp.coefs_[0].T, axes.ravel()):
    ax.matshow(coef.reshape(28, 28), cmap=plt.cm.gray, vmin=0.5 * vmin, vmax=0.5 * vmax)
    ax.set_xticks(())
    ax.set_yticks(())

plt.show()

In [ ]:
import numpy as np
from matplotlib import pyplot as plt
from matplotlib.colors import ListedColormap

from sklearn.datasets import make_circles, make_classification, make_moons
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler

In [ ]:
h = 0.02  # step size in the mesh

alphas = np.logspace(-1, 1, 5)

classifiers = []
names = []
for alpha in alphas:
    classifiers.append(
        make_pipeline(
            StandardScaler(),
            MLPClassifier(
                solver="lbfgs",
                alpha=alpha,
                random_state=1,
                max_iter=2000,
                early_stopping=True,
                hidden_layer_sizes=[10, 10],
            ),
        )
    )
    names.append(f"alpha {alpha:.2f}")

In [ ]:
X, y = make_classification(
    n_features=2, n_redundant=0, n_informative=2, random_state=0, n_clusters_per_class=1
)
rng = np.random.RandomState(2)
X += 2 * rng.uniform(size=X.shape)
linearly_separable = (X, y)

datasets = [
    make_moons(noise=0.3, random_state=0),
    make_circles(noise=0.2, factor=0.5, random_state=1),
    linearly_separable,
]

figure = plt.figure(figsize=(17, 9))
i = 1
# iterate over datasets
for X, y in datasets:
    # split into training and test part
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.4, random_state=42
    )

    x_min, x_max = X[:, 0].min() - 0.5, X[:, 0].max() + 0.5
    y_min, y_max = X[:, 1].min() - 0.5, X[:, 1].max() + 0.5
    xx, yy = np.meshgrid(np.arange(x_min, x_max, h), np.arange(y_min, y_max, h))

    # just plot the dataset first
    cm = plt.cm.RdBu
    cm_bright = ListedColormap(["#FF0000", "#0000FF"])
    ax = plt.subplot(len(datasets), len(classifiers) + 1, i)
    # Plot the training points
    ax.scatter(X_train[:, 0], X_train[:, 1], c=y_train, cmap=cm_bright)
    # and testing points
    ax.scatter(X_test[:, 0], X_test[:, 1], c=y_test, cmap=cm_bright, alpha=0.6)
    ax.set_xlim(xx.min(), xx.max())
    ax.set_ylim(yy.min(), yy.max())
    ax.set_xticks(())
    ax.set_yticks(())
    i += 1

    # iterate over classifiers
    for name, clf in zip(names, classifiers):
        ax = plt.subplot(len(datasets), len(classifiers) + 1, i)
        clf.fit(X_train, y_train)
        score = clf.score(X_test, y_test)

        # Plot the decision boundary. For that, we will assign a color to each
        # point in the mesh [x_min, x_max] x [y_min, y_max].
        if hasattr(clf, "decision_function"):
            Z = clf.decision_function(np.column_stack([xx.ravel(), yy.ravel()]))
        else:
            Z = clf.predict_proba(np.column_stack([xx.ravel(), yy.ravel()]))[:, 1]

        # Put the result into a color plot
        Z = Z.reshape(xx.shape)
        ax.contourf(xx, yy, Z, cmap=cm, alpha=0.8)

        # Plot also the training points
        ax.scatter(
            X_train[:, 0],
            X_train[:, 1],
            c=y_train,
            cmap=cm_bright,
            edgecolors="black",
            s=25,
        )
        # and testing points
        ax.scatter(
            X_test[:, 0],
            X_test[:, 1],
            c=y_test,
            cmap=cm_bright,
            alpha=0.6,
            edgecolors="black",
            s=25,
        )

        ax.set_xlim(xx.min(), xx.max())
        ax.set_ylim(yy.min(), yy.max())
        ax.set_xticks(())
        ax.set_yticks(())
        ax.set_title(name)
        ax.text(
            xx.max() - 0.3,
            yy.min() + 0.3,
            f"{score:.3f}".lstrip("0"),
            size=15,
            horizontalalignment="right",
        )
        i += 1

figure.subplots_adjust(left=0.02, right=0.98)
plt.show()